# MATH2504 Project 2 2022 Semester 2 Submission

[Assignment Instructions](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/project2.html)

[Github Repo](https://github.com/LimaoC/Limao-Chang__Tiarne-Graves-2504-2022-PROJECT2)


Student names: Limao Chang - 4678554, Tiarne Graves - 46974081

## Project Structure

In the root folder is `simulation_script.jl`, which imports and loads the `GeneralizedJacksonSim` module, as well as some other packages that are useful when working with `GeneralizedJacksonSim`.

The `src/` folder contains the source code for `GeneralizedJacksonSim`.

The `test/` folder contains tests which can be run in `test/runtests.jl`, as well as the four provided test scenarios, which can be found in `test/scenarios.jl`.

## Perspective Seminar

Due to the low quality of audio, this paragraph is in response to Dr Foeglein’s 2021 seminar: <br>
Dr Anna Foeglein knew that she wanted to be involved in simulation since she was a child, her interest being piqued when her mother brought home a computer. Anna studied a maths degree, majoring in pure mathematics and making some substantial contributions in her thesis that are still used today. Anna worked at the university for a while however was less interested in the tutoring aspect than she was in the software used to input and calculate students’ grades. She worked at simulation group for around 10 years before moving to the Integrated Logistics Company and AnyLogic where she currently works. She spends approximately 20% of her time selling software with AnyLogic, and the other 80% is spent working with the coal industry running simulations and models to organise coal shipment and delivery. <br>
There were many tools Dr Foeglein mentioned throughout her seminar including more code-based tools such as Simpy and SLX as well as more graphically based tools such as Jamsim. Even tools such as MATLAB Simulink are used, and which I found interesting given my current experience of MATLAB in mathematics courses at UQ. <br>
An interesting tip that I picked up from Anna’s talks are key things to form a good mathematical question, these things include being linked to a decision as well as an organisations goals, and being bounded, specific, and quantifiable. This links strongly to the concept of ‘mathematising’ that Anna introduced, an intriguing concept that means translating a regular question into one that can be mathematically answered.


## Task 1

**Features of amusement park that are not captured by model**:
- Amusement park attractions usually serve multiple people simultaneously, whereas the model assumes jobs can only be serviced one at a time
- People may cut in line and skip queues, so in reality it is not really a queue in the first-in, first-out (FIFO) sense

**Model assumptions that are unrealistic for the application of a theme park**:
- Assuming that queues (and by extension the theme park) have infinite capacity
- Assuming that people will unconditionally stay in a queue until they get to the front of the line - they may leave because the queue is too long, they want to go to another attraction, etc.
- Assuming that the time taken to move between attractions or leave the theme park is instant, as travelling between attractions takes time
- Service duration is independent from all other services - if people are at the theme park together as a party (e.g. a family travelling around the theme park), their service durations will not be independent from each other
- Arrival of jobs to nodes is exponentially distributed - the arrival rate will likely be higher during the day (more traffic) than in the evening, for example.

**If you were actually using this model to assess congestion at the park, would it be useful or not?**

While some model assumptions are unrealistic for application to a theme park, this model would still be a good starting point for congestion estimation. If a theme park can withstand the congestion according to this model it would be able to withstand realistic congestion as many of the model assumptions that are unrealistic put increased strain on the system.

## Task 2

The total steady state mean queue lengths are plotted against $\rho$* below for the four scenarios.

```julia
"""
    plot_total_ss_mean_queue_length(net::NetworkParameters)

Plots total steady state mean queue length as a function of ρ* for the given scenario.
"""
function plot_total_ss_mean_queue_length(net::NetworkParameters, scenario_number::Int64)
    ρ_star_values = 0.1:0.01:0.9
    total_ss_mean_queue_lengths = zeros(length(ρ_star_values))

    for (index, ρ_star) in enumerate(ρ_star_values)
        # adjust network parameters
        adjusted_scenario = set_scenario(net, ρ_star)
        ρ = compute_ρ(adjusted_scenario)

        total_ss_mean_queue_lengths[index] = sum(ρ ./ (1 .- ρ))
    end

    return plot(ρ_star_values,
                total_ss_mean_queue_lengths,
                xlabel="ρ",
                ylabel="Total Steady State Mean Queue Length",
                title="Scenario $scenario_number Theoretical")
end

p1 = plot_total_ss_mean_queue_length(scenario1, 1)
p2 = plot_total_ss_mean_queue_length(scenario2, 2)
p3 = plot_total_ss_mean_queue_length(scenario3, 3)
p4 = plot_total_ss_mean_queue_length(scenario4, 4)
        
plot(p1, p2, p3, p4, layout=(2, 2), legend=false, size=(800, 800))
```

<center>
    <img src="img/task2.png" width="640" />
</center>

## Task 3

**Simulation engine**: `src/GeneralizedJacksonSim.jl` contains the `GeneralizedJacksonSim` module, as well as the simulation engine used (`sim_net()` below).

```julia
# src/GeneralizedJacksonSim.jl
"""
A discrete event simulation engine for Open Generalized Jackson Networks.
"""
module GeneralizedJacksonSim

import Base: isless
using Accessors, DataStructures, Distributions, LinearAlgebra, Parameters, StatsBase,
    Random, Plots

include("network_parameters.jl")
include("customer.jl")
include("state.jl")
include("event.jl")

export NetworkParameters, QueueNetworkState, CustomerQueueNetworkState, sim_net,
    sim_net_customers, compute_ρ, maximal_alpha_scaling, set_scenario

"""
Runs a discrete event simulation of an Open Generalized Jackson Network `net`.

The simulation runs from time `0` to `max_time`.

Statistics about the total mean queue lengths are recorded from `warm_up_time` onwards
and the estimated value is returned.

This simulation does NOT keep individual customers' state, it only keeps the state which is
the number of items in each of the nodes.
"""
function sim_net(net::NetworkParameters;
                 state::State=QueueNetworkState(net), max_time::Int64=10^6,
                 warm_up_time::Int64=10^4, seed::Int64=42)
    Random.seed!(seed)

    # create priority queue and add standard events
    priority_queue = BinaryMinHeap{TimedEvent}()
    for q in 1:net.L
        if net.α_vector[q] > 0
            push!(priority_queue,
                TimedEvent(ExternalArrivalEvent(q), next_arrival_time(state, q)))
        end
    end
    push!(priority_queue, TimedEvent(EndSimEvent(), max_time))

    # set up queues integral for computing total mean queue length
    queues_integral = 0
    queue_total = 0
    time = 0.0
    last_time = 0.0

    """
    Records the queue integral of the given state at the given point in time.
    """
    function record_integral(time::Float64)
        if time >= warm_up_time
            queues_integral += sum(queue_total * (time - last_time))
        end
        last_time = time
    end

    record_integral(time)

    # simulation loop
    while true
        queue_total = sum(state.queues)
        # process the next upcoming event
        timed_event = pop!(priority_queue)
        time = timed_event.time
        new_timed_events = process_event(time, state, timed_event.event)

        # record queue length
        record_integral(time)

        # end sim if we've reached the EndOfSim event
        isa(timed_event.event, EndSimEvent) && break

        # add new spawned events to queue
        for nte in new_timed_events
            push!(priority_queue, nte)
        end
    end

    return queues_integral / (max_time - warm_up_time)
end
```

The other functionalities of the simulation engine (`NetworkParameters`, `State`s, `Event`s, and the functions related to each of them) have been separated into their own files, to keep the code nice and structured. Below are the files:

```julia
# src/network_parameters.jl
"""
This file contains the NetworkParameters implementation and related functionality.
"""

"""
    NetworkParameters

# Fields
- `L::Int64`: the dimension of the network (number of nodes)
- `α_vector::Vector{Float64}`: the external arrival rates α_i >= 0
- `μ_vector::Vector{Float64}`: the service rates μ_i > 0
- `P::Matrix{Float64}`: the L×L routing matrix P
- `c_s::Float64`: squared coefficient of variation of the service processes, defaults to 1.0
"""
@with_kw struct NetworkParameters
    L::Int64
    α_vector::Vector{Float64}
    μ_vector::Vector{Float64}
    P::Matrix{Float64}
    c_s::Float64 = 1.0
end

"""
    compute_ρ(net::NetworkParameters)

Computes the vector of ρ values for a given set of network parameters.
"""
function compute_ρ(net::NetworkParameters)
    λ = (I - net.P') \ net.α_vector  # solve traffic equations
    return λ ./ net.μ_vector  # vector of ρ values
end

"""
Computes the maximal value by which we can scale the network's α_vector and be stable.
"""
function maximal_alpha_scaling(net::NetworkParameters)
    λ_base = (I - net.P') \ net.α_vector  # solve the traffic equations
    ρ_base = λ_base ./ net.μ_vector  # determine the load ρ  
    return minimum(1 ./ ρ_base)
end

"""
    set_scenario(net::NetworkParameters, ρ::Float64, c_s::Float64 = 1.0)

Adjusts the network parameters to the desired ρ⋆ and c_s.
"""
function set_scenario(net::NetworkParameters, ρ::Float64, c_s::Float64=1.0)
    (ρ ≤ 0 || ρ ≥ 1) && error("ρ is out of range")
    max_scaling = maximal_alpha_scaling(net)
    net = @set net.α_vector = net.α_vector * max_scaling * ρ
    net = @set net.c_s = c_s
    return net
end

# src/state.jl
"""
This file contains the State implementation and related functionality.
"""

"""A generic abstract State type."""
abstract type State end

"""
    QueueNetworkState

# Fields
- `queues::Vector{Int64}`: vector of number of customers in each queue
- `arrivals::Vector{Int64}`: vector of number of arrivals to each queue
- `num_queues::Int64`: number of queues, equivalent to `length(queues)`
- `net::NetworkParameters`: the parameters of the network
"""
mutable struct QueueNetworkState <: State
    queues::Vector{Int64}
    arrivals::Vector{Int64}
    num_queues::Int64
    net::NetworkParameters

    # Inner constructor for a given scenario's parameters
    function QueueNetworkState(net::NetworkParameters)
        new(zeros(Int64, net.L), zeros(Int64, net.L), net.L, net)
    end
end

"""
A wrapper around a Gamma distribution with desired rate (inverse of shape) and SCV.
"""
rate_scv_gamma(rate::Float64, scv::Float64) = Gamma(1/scv, scv/rate)

"""
    next_arrival_time(s::State, q::Int64)

Generates the next external arrival time for the `q`th server. The duration of time between
external arrivals is exponentially distributed with mean 1 / s.net.α_vector[q].
"""
next_arrival_time(s::State, q::Int64) = rand(Exponential(1/s.net.α_vector[q]))

"""
    next_service_time(s::State, q::Int64)

Generates the next service time for the `q`th server. The service duration is gamma
distributed.
"""
next_service_time(s::State, q::Int64) = rand(rate_scv_gamma(s.net.μ_vector[q], s.net.c_s))

# src/event.jl
"""
This file contains the Event implementation and related functionality.
"""

"""A generic abstract Event type."""
abstract type Event end

"""
An event that signifies an arrival from outside the system to the server `q`.
"""
struct ExternalArrivalEvent <: Event
    q::Int64
end

"""An event that signifies the end of a service at the `q`th server."""
struct EndOfServiceEvent <: Event
    q::Int64
end

"""An event that ends the simulation."""
struct EndSimEvent <: Event end

"""An event that prints a log of the current simulation state."""
struct LogStateEvent <: Event end

"""
    TimedEvent

# Fields
- `event::Event`: the stored event
- `time::Float64`: the time at which the event takes place (starts)
"""
struct TimedEvent
    event::Event
    time::Float64
end

"""
Returns true if the first event takes place earlier than the second event.
"""
isless(te1::TimedEvent, te2::TimedEvent) = te1.time < te2.time

function process_event(time::Float64, state::State, event::Event) end

"""
    process_event(time::Float64, state::QueueNetworkState, event::ExternalArrivalEvent)

Process an arrival event from outside the system, and spawns a list of events that occur as
a consequence of this arrival.

On arrival, if the server is free (no jobs in the buffer/queue), the job starts to receive
service. If the server is busy, the job queues for service and waits for its turn.

The time between external arrival events for a given server is exponentially distributed,
and the service duration is gamma distributed.
"""
function process_event(time::Float64, state::QueueNetworkState, event::ExternalArrivalEvent)
    q = event.q
    state.queues[q] += 1  # add to queue
    state.arrivals[q] += 1  # record arrival
    new_timed_events = TimedEvent[]

    # prepare next external arrival for this particular server
    push!(new_timed_events,
        TimedEvent(ExternalArrivalEvent(q), time + next_arrival_time(state, q)))

    # start serving this job if it is the only one in the queue
    if state.queues[q] == 1
        push!(new_timed_events,
            TimedEvent(EndOfServiceEvent(q), time + next_service_time(state, q)))
    end
    return new_timed_events
end

"""
    process_event(time::Float64, state::QueueNetworkState, event::EndOfServiceEvent)

Process an end-of-service event, and spawns a list of events that occur as a consequence of
this end of service.

When a job completes service at a buffer, it either leaves the system, or moves to another
buffer (both happen immediately). After completing service in server i, a job moves to
server j with probability P[i, j], where P is the routing matrix.
"""
function process_event(time::Float64, state::QueueNetworkState, event::EndOfServiceEvent)
    q = event.q
    state.queues[q] -= 1  # remove from queue
    @assert state.queues[q] >= 0
    new_timed_events = TimedEvent[]

    # if there is another customer in the queue, start serving them
    if state.queues[q] > 0
        service_time = next_service_time(state, q)
        push!(new_timed_events, TimedEvent(EndOfServiceEvent(q), time + service_time))
    end

    # simulate the next location for this job; indices 1:L are the probabilities of moving
    # to another server in the system, and the last index is the probability of exiting
    # the system
    L = state.net.L
    next_loc_weights = state.net.P[q, :]
    push!(next_loc_weights, 1 - sum(next_loc_weights))
    @assert sum(next_loc_weights) == 1
    next_loc = sample(1:L+1, Weights(next_loc_weights))

    if next_loc <= L
        state.queues[next_loc] += 1  # job is staying in the system
        state.arrivals[next_loc] += 1  # record arrival

        # start serving job if it is the only one in the queue
        if state.queues[next_loc] == 1
            service_time = next_service_time(state, next_loc)
            push!(new_timed_events,
                TimedEvent(EndOfServiceEvent(next_loc), time + service_time))
        end
    end
    return new_timed_events
end
```

**Test 1**: the code for test 1 can be found in `test/task3_test1.jl`. We decided to advantage of multi-threading for this test, since each simulation can be run independently, and their results can be stored without affecting any other simulations (see the source code for more details). To make use of multi-threading, you have to run Julia with the `-t` or `--threads` option, and specify the number of threads you want to utilise, e.g. `julia -t 8` to run Julia with 8 threads. The test can then by executed using:
```julia
task3_test1(
    [scenario1, scenario2, scenario3, scenario4],  # vector of all scenarios to test
    verbose=true,  # whether to print out progress messages as the simulations are run
    multithreaded=true  # whether to utilise multi-threading (julia needs to be run with
                        # `julia -t n` where n is the number of threads)
)
```

An example output is given below, for `max_time` ranging from 10^3 to 10^5 (10^6 is unreasonable for scenario 4 - it took around half an hour to run scenario 4 with `max_time = 10^6`). In every scenario, the absolute relative errors seem to decrease as `max_time` gets larger.

<center>
    <img src="img/task3test1.png" width="640" />
</center>

**Test 2**: the code for test 2 can be found in `test/task3_test2.jl`. In order to record the number of arrivals $A_i(t)$, we added an `arrivals` field to the `QueueNetworkState` struct. So at the end of the simulation, the `state` will be populated with the total number of arrivals to each node, and we can access this through `state.arrivals`. However, we need to be able to have a reference to `state` from outside the `sim_net()` function to be able to calculate the simulated mean number of arrivals in the test. One solution is to have `sim_net()` return the mean number of arrivals as well, but this isn't really a clean solution and breaks compatibility with previous tests and scripts that rely on `sim_net()` only returning one float (total mean queue length). A better solution would be to add `state` as a keyword argument to `sim_net()`:

```julia
# src/GeneralizedJacksonSim.jl
function sim_net(net::NetworkParameters;
                 state::State=QueueNetworkState(net), max_time::Int64=10^6,
                 warm_up_time::Int64=10^4, seed::Int64=42)::Float64
    
# src/state.jl
mutable struct QueueNetworkState <: State
    queues::Vector{Int}
    arrivals::Vector{Int}
    num_queues::Int
    net::NetworkParameters

    # Inner constructor for a given scenario's parameters
    function QueueNetworkState(net::NetworkParameters)
        new(zeros(Int64, net.L), zeros(Int64, net.L), net.L, net)
    end
end
```

Now if we pass in a `state` to `sim_net()`, we have a reference to the final state of the simulation when it ends, and we can query `state.arrivals` after running `sim_net()` to get the total number of arrivals to each node. Importantly, this doesn't break compatibility with other uses of `sim_net()`, since state will default to `QueueNetworkState(net)` if there isn't a state passed.

```julia
# test/task3_test2.jl
"""
This test compares the average number of arrivals to each node in the system from the
simulation, `sim_net()`, against the theoretical average number of arrivals.
"""

"""
    task3_test2(scenarios::Vector{NetworkParameters}; max_time::Int64=10^6,
                digits::Int64=3)

Prints the simulated average number of arrivals (rounded to `digits` decimal places) and
the theoretical average number of arrivals. If a scenario is unstable, it is adjusted to be
stable by setting its ρ* to `ρ_star`.
"""
function task3_test2(scenarios::Vector{NetworkParameters};
                     max_time::Int64=10^5, digits::Int64=3, ρ_star=0.8,
                     c_s_values::Vector{Float64}=[0.1, 0.5, 1.0, 2.0, 4.0])
    for (index, scenario) in enumerate(scenarios)
        println("Simulating scenario $index...")
        # make scenario stable if it is not stable
        if maximum(compute_ρ(scenario)) > 1
            scenario = set_scenario(scenario, ρ_star)
        end

        for c_s in c_s_values
            println("   c_s = $c_s:")
            scenario = @set scenario.c_s = c_s
            # pass in a state to sim_net() so we have a reference to the state to calculate the
            # mean number of arrivals
            state = QueueNetworkState(scenario)
            _ = sim_net(scenario, state=state, max_time=max_time)

            # calculate mean number of arrivals
            mean_num_arrivals = state.arrivals ./ max_time
            theoretical_mean_num_arrivals = (I - scenario.P') \ scenario.α_vector
            sum_of_squares = sum((mean_num_arrivals - theoretical_mean_num_arrivals).^2)

            if length(mean_num_arrivals) > 10
                mean_num_arrivals = mean_num_arrivals[begin:10]
                theoretical_mean_num_arrivals = theoretical_mean_num_arrivals[begin:10]
                println("       (truncated to first 10 nodes for readability)")
            end

            println("       Simulated mean number of arrivals  : " *
                    "$(round.(mean_num_arrivals, digits=digits))")
            println("       Theoretical mean number of arrivals: " *
                    "$(round.(theoretical_mean_num_arrivals, digits=digits))")
            println("       Sum of squared differences         : $sum_of_squares")
        end
    end
end
```

Running the test for all 4 scenarios gives
```
Simulating scenario 1...
   c_s = 0.1:
       Simulated mean number of arrivals  : [0.498, 0.498, 0.498]
       Theoretical mean number of arrivals: [0.5, 0.5, 0.5]
       Sum of squared differences         : 1.6427000000000122e-5
   c_s = 0.5:
       Simulated mean number of arrivals  : [0.498, 0.497, 0.497]
       Theoretical mean number of arrivals: [0.5, 0.5, 0.5]
       Sum of squared differences         : 1.8900500000000183e-5
   c_s = 1.0:
       Simulated mean number of arrivals  : [0.497, 0.497, 0.497]
       Theoretical mean number of arrivals: [0.5, 0.5, 0.5]
       Sum of squared differences         : 2.84592000000005e-5
   c_s = 2.0:
       Simulated mean number of arrivals  : [0.497, 0.497, 0.497]
       Theoretical mean number of arrivals: [0.5, 0.5, 0.5]
       Sum of squared differences         : 2.5404300000000417e-5
   c_s = 4.0:
       Simulated mean number of arrivals  : [0.5, 0.5, 0.5]
       Theoretical mean number of arrivals: [0.5, 0.5, 0.5]
       Sum of squared differences         : 6.911999999999437e-7
Simulating scenario 2...
   c_s = 0.1:
       Simulated mean number of arrivals  : [0.711, 0.711, 0.711]
       Theoretical mean number of arrivals: [0.714, 0.714, 0.714]
       Sum of squared differences         : 3.965705510204108e-5
   c_s = 0.5:
       Simulated mean number of arrivals  : [0.715, 0.715, 0.715]
       Theoretical mean number of arrivals: [0.714, 0.714, 0.714]
       Sum of squared differences         : 3.0069836734692716e-6
   c_s = 1.0:
       Simulated mean number of arrivals  : [0.711, 0.711, 0.711]
       Theoretical mean number of arrivals: [0.714, 0.714, 0.714]
       Sum of squared differences         : 3.3380969387756e-5
   c_s = 2.0:
       Simulated mean number of arrivals  : [0.717, 0.717, 0.717]
       Theoretical mean number of arrivals: [0.714, 0.714, 0.714]
       Sum of squared differences         : 2.5717126530611948e-5
   c_s = 4.0:
       Simulated mean number of arrivals  : [0.712, 0.712, 0.712]
       Theoretical mean number of arrivals: [0.714, 0.714, 0.714]
       Sum of squared differences         : 1.2849340816326528e-5
Simulating scenario 3...
   c_s = 0.1:
       Simulated mean number of arrivals  : [0.801, 0.8, 0.798, 0.799, 0.799]
       Theoretical mean number of arrivals: [0.8, 0.8, 0.8, 0.8, 0.8]
       Sum of squared differences         : 9.330100000000268e-6
   c_s = 0.5:
       Simulated mean number of arrivals  : [0.801, 0.799, 0.797, 0.797, 0.797]
       Theoretical mean number of arrivals: [0.8, 0.8, 0.8, 0.8, 0.8]
       Sum of squared differences         : 2.7853900000001146e-5
   c_s = 1.0:
       Simulated mean number of arrivals  : [0.798, 0.797, 0.8, 0.799, 0.8]
       Theoretical mean number of arrivals: [0.8, 0.8, 0.8, 0.8, 0.8]
       Sum of squared differences         : 1.4997300000000487e-5
   c_s = 2.0:
       Simulated mean number of arrivals  : [0.802, 0.803, 0.801, 0.801, 0.803]
       Theoretical mean number of arrivals: [0.8, 0.8, 0.8, 0.8, 0.8]
       Sum of squared differences         : 2.235429999999947e-5
   c_s = 4.0:
       Simulated mean number of arrivals  : [0.801, 0.8, 0.801, 0.802, 0.802]
       Theoretical mean number of arrivals: [0.8, 0.8, 0.8, 0.8, 0.8]
       Sum of squared differences         : 9.980999999999514e-6
Simulating scenario 4...
   c_s = 0.1:
       (truncated to first 10 nodes for readability)
       Simulated mean number of arrivals  : [0.397, 0.404, 0.393, 0.397, 0.397, 0.395, 0.398, 0.422, 0.381, 0.407]
       Theoretical mean number of arrivals: [0.399, 0.399, 0.395, 0.398, 0.395, 0.393, 0.396, 0.417, 0.381, 0.408]
       Sum of squared differences         : 0.0003935084561040076
   c_s = 0.5:
       (truncated to first 10 nodes for readability)
       Simulated mean number of arrivals  : [0.398, 0.397, 0.395, 0.397, 0.391, 0.391, 0.391, 0.421, 0.38, 0.408]
       Theoretical mean number of arrivals: [0.399, 0.399, 0.395, 0.398, 0.395, 0.393, 0.396, 0.417, 0.381, 0.408]
       Sum of squared differences         : 0.00038710725810373796
   c_s = 1.0:
       (truncated to first 10 nodes for readability)
       Simulated mean number of arrivals  : [0.399, 0.403, 0.395, 0.398, 0.395, 0.392, 0.394, 0.416, 0.381, 0.406]
       Theoretical mean number of arrivals: [0.399, 0.399, 0.395, 0.398, 0.395, 0.393, 0.396, 0.417, 0.381, 0.408]
       Sum of squared differences         : 0.0004263142629795542
   c_s = 2.0:
       (truncated to first 10 nodes for readability)
       Simulated mean number of arrivals  : [0.402, 0.399, 0.392, 0.397, 0.397, 0.394, 0.396, 0.417, 0.383, 0.412]
       Theoretical mean number of arrivals: [0.399, 0.399, 0.395, 0.398, 0.395, 0.393, 0.396, 0.417, 0.381, 0.408]
       Sum of squared differences         : 0.0004501958361770971
   c_s = 4.0:
       (truncated to first 10 nodes for readability)
       Simulated mean number of arrivals  : [0.397, 0.4, 0.393, 0.398, 0.395, 0.39, 0.392, 0.422, 0.386, 0.407]
       Theoretical mean number of arrivals: [0.399, 0.399, 0.395, 0.398, 0.395, 0.393, 0.396, 0.417, 0.381, 0.408]
       Sum of squared differences         : 0.0004319871998824227
```

## Task 4

The code below generates simulated mean queue length for each service time at each p* value. The function takes a specific network (i.e. scenario 1-4). The array, plotting_data, stores 5 vectors, one for each service time.

```julia
c_s_values = [0.1,0.5,1.0,2.0,4.0]
        
#generates the y-values for each c_s value for each scenario
function plot_mean_queue_length_c_s_values(net::NetworkParameters, ρ_star_values;
                                           max_time=10^3,
                                           warm_up_time=10)
    plotting_data = Vector{Vector{Float64}}()
    # forming data for each c_s value
    for c_s in c_s_values
        simulated_total_mean_queue_lengths = zeros(length(ρ_star_values))
        # forming data for each p*
        for (index, ρ_star) in enumerate(ρ_star_values)
            # adjust network parameters
            adjusted_net = set_scenario(net, ρ_star, c_s)
            simulated = sim_net(adjusted_net, max_time=max_time, warm_up_time=warm_up_time)
            # get total steady state mean queue length
            simulated_total_mean_queue_lengths[index] = simulated
        end
        # add vector of simulated mean queue lengths for each c_s value into overall vector
        push!(plotting_data, simulated_total_mean_queue_lengths)
    end
    return plotting_data
end
```

To plot the required graphs, the above function is called in the below function `plotting_service_times()`, which allows the 5 varying service times for each scenario to be plotted on one figure, by extracting data from the plotting_data array.

```julia
#plots graphs for each scenario with varying c_s_value values displayed
function plotting_c_s_values(net::NetworkParameters, scenario_number::Int64)
    ρ_star_values = 0.01:0.01:0.9
    println("Simulating scenario $scenario_number with varying c_s values...")
    data = plot_mean_queue_length_c_s_values(net, ρ_star_values, max_time=10^3,
                                             warm_up_time=10)

    return plot(
        ρ_star_values,
        data,
        xlabel="ρ",
        ylabel="Total Mean Queue Length",
        title="Scenario $scenario_number Simulation with varying c_s values",
        labels=["0.1" "0.5" "1.0" "2.0" "4.0"])
end

#plotting initial servcice time graphs
p1 = plotting_c_s_values(scenario1, 1)
p2 = plotting_c_s_values(scenario2, 2)
p3 = plotting_c_s_values(scenario3, 3)
p4 = plotting_c_s_values(scenario4, 4)

display(plot(p1,p2,p3,p4, layout=(4, 1), legend=true, size=(900, 900)))
```

This results in the below graphs, it can be seen that higher service times e.g. 4.0 result in a mean total queue length compared to lower service times e.g. 0.1

<center>
    <img src="img/plots_of_various_c_s_values.png" width="640" />
</center>

Confidence bounds for these curves were also generated using the code below, where 100 different points were generated for each p* by using a different seed for each generation as can be seen in the for loops below. It is important to note however, that given each point is generated 100 times for increased accuracy, this section of code take a while to run to completition.

```julia
#calculates and returns plot of confidence bounds for each scenario and c_s_value value
function confidence_bounds(net::NetworkParameters, scenario_number::Int64,
                           c_s_value::Float64,max_time=10^3, warm_up_time=10)
    println("Simulating scenario $scenario_number confidence bounds for c_s = $c_s_value...")
    ρ_star_values = 0.01:0.01:0.9
    data = []
    mean_data = []
    top_percentile = []
    bottom_percentile = []
    for (index, ρ_star) in enumerate(ρ_star_values)
        adjusted_net = set_scenario(net, ρ_star, c_s_value)
        #generates simulated data for each p* 100 times at 100 varying seeds
        for seed in 1:100
            simulated = sim_net(adjusted_net, max_time=max_time,
                                warm_up_time=warm_up_time, seed=seed)
            push!(data, simulated)
        end
        #pushes in the mean and upper and lower confidence bounds data
        push!(mean_data, mean(data))
        push!(top_percentile, percentile(data,95))
        push!(bottom_percentile, percentile(data,5))
        end
    #plots confidence bounds graph
    return plot(ρ_star_values,mean_data; ribbon = (bottom_percentile,top_percentile), xlabel="ρ",
        ylabel="Total Mean Queue Length",
        title="Scenario $scenario_number with $c_s_value c_s",
        titlefontsize=10,
        labels=["mean" "95th percentile" "5th percentile"])
end

# plotting confidence bound graphs
plots = Vector()
for (index,scenario) in enumerate(scenarios)
    for c_s_value in c_s_values
        push!(plots, confidence_bounds(scenario, index, c_s_value))
    end
end
display(plot(plots..., layout=(5, 4), legend=true, size=(1100, 1100)))
```

The resulting confidence bounds for each scenario were as below:

<center>
    <img src="img/confidence_bounds.png" width="640" />
</center>






## Task 5

`sim_net_customers()` is largely the same as `sim_net()`, except for a few changes. The state used is a `CustomerQueueNetworkState`, which implements `queues` as a vector of `Queue{Customer}`s instead of a vector of `Int64`s, and keeps track of all departed customers in the field `departed_customers`. That is, we keep track of the individual customers rather than just the number of customers in each queue.

```julia
# src/GeneralizedJacksonSim.jl
"""
Runs a discrete event simulation of an Open Generalized Jackson Network `net`.

The simulation runs from time `0` to `max_time`.

Statistics about the total mean queue lengths are recorded from `warm_up_time` onwards
and the estimated value is returned.

This simulation keeps track of individual customer states - specifically, the time at which
they enter the system and the time at which they exit.
"""
function sim_net_customers(net::NetworkParameters;
                           state::State=CustomerQueueNetworkState(net),
                           max_time::Int64=10^6, warm_up_time::Int64=10^4,
                           seed::Int64=42)::Float64
    Random.seed!(seed)

    # create priority queue and add standard events
    priority_queue = BinaryMinHeap{TimedEvent}()
    for q in 1:net.L
        if net.α_vector[q] > 0
            push!(priority_queue,
                TimedEvent(CustomerExternalArrivalEvent(q), next_arrival_time(state, q)))
        end
    end
    push!(priority_queue, TimedEvent(EndSimEvent(), max_time))

    # set up queues integral for computing total mean queue length
    queues_integral = 0
    queue_total = 0
    time = 0.0
    last_time = 0.0

    """
    Records the queue integral of the given state at the given point in time.
    """
    function record_integral(time::Float64)
        if time >= warm_up_time
            queues_integral += sum(queue_total * (time - last_time))
            # queues_integral += sum(map((queue) -> length(queue), state.queues) *
            #     (time - last_time))
        end
        last_time = time
    end

    record_integral(time)

    # simulation loop
    while true
        queue_total = sum(map((queue) -> length(queue), state.queues))
        # process the next upcoming event
        timed_event = pop!(priority_queue)
        time = timed_event.time
        new_timed_events = process_event(time, state, timed_event.event)

        # record mean queue length
        record_integral(time)

        # end sim if we've reached the EndOfSim event
        isa(timed_event.event, EndSimEvent) && break

        # add new spawned events to queue
        for nte in new_timed_events
            push!(priority_queue, nte)
        end
    end

    return queues_integral / (max_time - warm_up_time)
end

# src/customer.jl
"""
This file contains the Customer implementation and related functionality.
"""

"""
    Customer

# Fields
`arrival::Float64`: time of arrival into the system
`departure::Float64`: time of departure from the system. equal to -1.0 if customer is still
                      in the system.
"""
mutable struct Customer
    arrival::Float64
    departure::Float64
end

# src/state.jl
"""
    CustomerQueueNetworkState

# Fields
- `queues::Vector{Queue{Customer}}`: vector of queues which store Customers
- `arrivals::Vector{Int64}`: vector of number of arrivals to each queue
- `num_queues::Int64`: number of queues, equivalent to `length(queues)`
- `net::NetworkParameters`: the parameters of the network
"""
mutable struct CustomerQueueNetworkState <: State
    queues::Vector{Queue{Customer}}
    departed_customers::Vector{Customer}
    arrivals::Vector{Int64}
    num_queues::Int64
    net::NetworkParameters

    # Inner constructor for a given scenario's parameters
    function CustomerQueueNetworkState(net::NetworkParameters)
        queues = Vector{Queue{Customer}}()
        for _ in 1:net.L
            push!(queues, Queue{Customer}())
        end
        new(queues, Vector{Customer}(), zeros(Int64, net.L), net.L, net)
    end
end
```

`record_integral()` in `sim_net_customers()` also needs to be changed; we need to map the vector of `Queue{Customer}`s to a vector of the length of each queue:

```julia
# src/GeneralizedJacksonSim.jl
function record_integral(time::Float64, state::State)
    if time >= warm_up_time
        queues_integral += (map((queue) -> length(queue), state.queues) *
            (time - last_time))
    end
    last_time = time
end
```

The biggest changes are in the `process_event()` functions. We introduce the following new `Event` types for `Customer`s:

```julia
# src/event.jl
struct CustomerExternalArrivalEvent <: Event
    q::Int64
end
struct CustomerEndOfServiceEvent <: Event
    q::Int64
end
```

On arrival into the system, a `Customer` is created with the corresponding `arrival` time, and `-1.0` as the `departure` time (`-1.0` signifying that the customer hasn't left the system yet).

```julia
# src/event.jl
function process_event(time::Float64, state::CustomerQueueNetworkState,
                       event::CustomerExternalArrivalEvent)
    q = event.q
    enqueue!(state.queues[q], Customer(time, -1.0))  # add new customer to queue
    state.arrivals[q] += 1  # record arrival
    new_timed_events = TimedEvent[]

    # prepare next external arrival for this particular server
    push!(new_timed_events,
        TimedEvent(CustomerExternalArrivalEvent(q), time + next_arrival_time(state, q)))

    # start serving this job if it is the only one in the queue
    if length(state.queues[q]) == 1
        push!(new_timed_events,
            TimedEvent(CustomerEndOfServiceEvent(q), time + next_service_time(state, q)))
    end
    return new_timed_events
end
```

When a customer finishes at a server, we record when it leaves the system in the `customer`'s `departure` field, and add the customer to the `state`'s `departed_customers` field. If the customer is just moving to another server, the logic is the same as for an `EndOfServiceEvent`.

```julia
# src/event.jl
function process_event(time::Float64, state::CustomerQueueNetworkState,
                       event::CustomerEndOfServiceEvent)
    q = event.q
    customer = dequeue!(state.queues[q])  # remove from queue
    @assert length(state.queues[q]) >= 0
    new_timed_events = TimedEvent[]

    # if there is another customer in the queue, start serving them
    if length(state.queues[q]) > 0
        push!(new_timed_events,
            TimedEvent(CustomerEndOfServiceEvent(q), time + next_service_time(state, q)))
    end

    # simulate the next location for this job; indices 1:L are the probabilities of moving
    # to another server in the system, and the last index is the probability of exiting
    # the system
    L = state.net.L
    next_loc_weights = state.net.P[q, :]
    push!(next_loc_weights, 1 - sum(next_loc_weights))
    @assert sum(next_loc_weights) == 1
    next_loc = sample(1:L+1, Weights(next_loc_weights))

    if next_loc <= L
        enqueue!(state.queues[next_loc], customer)  # job is staying in the system
        state.arrivals[next_loc] += 1  # record arrival

        # start serving job if it is the only one in the queue
        if length(state.queues[next_loc]) == 1
            service_time = next_service_time(state, next_loc)
            push!(new_timed_events,
                TimedEvent(CustomerEndOfServiceEvent(next_loc), time + service_time))
        end
    else # job leaving the system 
        customer.departure = time
        push!(state.departed_customers, customer)
    end
    return new_timed_events
end
```

We can use the `state`'s `departed_customers` vector to estimate the Q1, median, and Q3 sojourn time like so:

```julia
function estimate_sojourn_time(scenarios::Vector{NetworkParameters}; max_time::Int64=10^6,
                               warm_up_time::Int64=10^4, ρ_star::Float64=0.8,
                               c_s_values::Vector{Float64}=[0.5, 1.0, 2.0])
    for (index, scenario) in enumerate(scenarios)
        println("Scenario $index:")
        scenario = set_scenario(scenario, ρ_star)
        for c_s in c_s_values
            println("    c_s = $c_s:")
            scenario = @set scenario.c_s = c_s
            state = CustomerQueueNetworkState(scenario)
            sim_net_customers(scenario, state=state, max_time=max_time, warm_up_time=warm_up_time)

            sojourn_times = map((c) -> (c.departure - c.arrival), state.departed_customers)
            quartiles = nquantile(sojourn_times, 4)
            println("        Estimated Q1: $(quartiles[2])")
            println("        Estimated Q2: $(quartiles[3])")
            println("        Estimated Q3: $(quartiles[4])")
        end
    end
end

scenarios = [scenario1, scenario2, scenario3, scenario4]
estimate_sojourn_time(scenarios, max_time=10^5, warm_up_time=10^3)
```

In a formatted table:

| Scenario | c_s value | Estimated Q1 | Estimated Q2 | Estimated Q3 |
| -------- | --------- | ------------ | ------------ | ------------ |
| Scenario 1 | 0.5 | 6.079422459621355 | 9.248509342971374 | 13.40359156129631
| Scenario 1 | 1.0 | 8.673130520382983 | 13.59239267126759 | 20.093707535765134
| Scenario 1 | 2.0 | 13.046784005859536 | 20.91962903278545 | 30.70727474848536
| Scenario 2 | 0.5 | 7.366778173680359 | 11.891636960881442 | 19.06973305059364
| Scenario 2 | 1.0 | 10.33712767445104 | 16.378538413166098 | 26.12008817997412
| Scenario 2 | 2.0 | 15.240999229907175 | 24.794787939918024 | 40.30262643196693
| Scenario 3 | 0.5 | 0.9189720012013822 | 3.3836975322738 | 7.7106822436035145
| Scenario 3 | 1.0 | 0.951647432105915 | 3.730264990048454 | 8.924551431995496
| Scenario 3 | 2.0 | 0.9975650895648869 | 4.607887606838631 | 11.716368674995465
| Scenario 4 | 0.5 | 1.2602543030243396 | 2.819662968560806 | 5.843733932077157
| Scenario 4 | 1.0 | 1.1594554236075965 | 3.0126887688547868 | 6.698496126526152
| Scenario 4 | 2.0 | 0.9801621326832901 | 3.3684753331908723 | 8.437446720914522